In [ ]:
!pip install -Uqq pyts fastbook binance-connector pandas_ta

In [ ]:
# Import necessary libraries
import datetime as dt
import os
import warnings
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import pandas as pd
import pandas_ta as ta
import pywt
import pywt.data
import re
from pyts.image import GramianAngularField
from PIL import Image as im
from binance.spot import Spot
%matplotlib inline 

import fastbook
fastbook.setup_book()
from fastbook import *
import matplotlib.image

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pywt.wavelist('sym')

In [ ]:
SYMBOL = 'BTCUSDT'
INTERVAL = '1m'
INPUT_SIZE = 30
RAW_INPUT_SIZE = 100
THRESHOLD = 0.97
TRADING_PERCENT = 0.1
INITIAL_USD_BALANCE = 1000
STOP_PROFIT = 0.004
STOP_LOSS = 0.004
ORDER_LIFE = 5
TALIB_OFFSET = 33

In [ ]:
def to_dataframe(data):
   np_data = np.array(data)
   df = pd.DataFrame(data=np_data[:, 1:6],
                     index=np_data[:, 0],
                     columns=np_data[0, 1:6])
   df.columns = ['open', 'high', 'low', 'close', 'volume']
   df['open'] = pd.to_numeric(df['open'])
   df['high'] = pd.to_numeric(df['high'])
   df['low'] = pd.to_numeric(df['low'])
   df['close'] = pd.to_numeric(df['close'])
   df['volume'] = pd.to_numeric(df['volume'])


In [ ]:
rest_client = Spot()
START_TIME = 0
def prepare_dataset():
    first_data = rest_client.klines(SYMBOL, INTERVAL, limit=500)
    df = to_dataframe(first_data)
    startTime = 4
    #while endTime > START_TIME:
    #    data = rest_client.klines(SYMBOL, INTERVAL, limit=500)

In [ ]:
wavelet_type = 'sym15'
w = pywt.Wavelet(wavelet_type)
def denoise(data):
    if len(data) > 0:
        maxlev = pywt.dwt_max_level(len(data), w.dec_len)
        coeffs = pywt.wavedec(data, wavelet_type, level=maxlev)
        coeffs[-1] = np.zeros_like(coeffs[-1])
        datarec = pywt.waverec(coeffs, wavelet_type)
        return datarec
    else:
        return data

In [ ]:
BTC = pd.read_csv("../input/bitcoin-btcusdt/Bitcoin_BTCUSDT.csv")
df = BTC.iloc[-10000:].reset_index(drop=True)

In [ ]:

def calc_label(dataset, position):
    price = dataset['close'].to_numpy()[position]
    stop_loss = STOP_LOSS * price
    stop_profit = STOP_PROFIT * price
    label = "wait"
    bearish_stop_loss = price + stop_loss
    bearish_stop_profit = price - stop_profit
    bullish_stop_loss = price - stop_loss
    bullish_stop_profit = price + stop_profit
    for i in range(1, ORDER_LIFE+1):
        max_price = dataset['high'].to_numpy()[position+i]
        low_price = dataset['low'].to_numpy()[position+i]
        if low_price > bullish_stop_loss:
            if max_price >= bullish_stop_profit:
                label = "buy"
                break
        else:
            break
    for i in range(1, ORDER_LIFE+1):
        max_price = dataset['high'].to_numpy()[position+i]
        low_price = dataset['low'].to_numpy()[position+i]
        if max_price < bearish_stop_loss:
            if low_price <= bearish_stop_profit:
                label = "sell"
                break
        else:
            break
    return label
        
       

    

In [ ]:
L = len(df['close'])
df_labels = [None]*L
for i in range(0, L-ORDER_LIFE):
    df_labels[i] = calc_label(df, i)

df_labeled_raw = df.copy()
df_labeled_raw['label'] = df_labels

In [ ]:
df_labeled_raw[150:300]

In [ ]:


df = df_labeled_raw.iloc[:-ORDER_LIFE].reset_index(drop=True).copy()

In [ ]:
DS_LENGTH = len(df['close'])
df

In [ ]:
input_open = [None]*DS_LENGTH
input_high = [None]*DS_LENGTH
input_low = [None]*DS_LENGTH
input_close = [None]*DS_LENGTH
input_volume = [None]*DS_LENGTH
input_timestamp = [None]*DS_LENGTH
for i in range(0, DS_LENGTH):
    if i + 1 >= RAW_INPUT_SIZE:
        input_open[i] = df.iloc[i+1-RAW_INPUT_SIZE:i+1].open.copy()
        input_high[i] = df.iloc[i+1-RAW_INPUT_SIZE:i+1].high.copy()
        input_low[i] = df.iloc[i+1-RAW_INPUT_SIZE:i+1].low.copy()
        input_close[i] = df.iloc[i+1-RAW_INPUT_SIZE:i+1].close.copy()
        input_volume[i] = df.iloc[i+1-RAW_INPUT_SIZE:i+1].volume.copy()
        input_timestamp[i] = df.iloc[i+1-RAW_INPUT_SIZE:i+1].timestamp.copy().reset_index(drop=True)
df['input_open'] = input_open
df['input_high'] = input_high
df['input_low'] = input_low
df['input_close'] = input_close
df['input_volume'] = input_volume
df['input_timestamp'] = input_timestamp
df = df.iloc[RAW_INPUT_SIZE:].copy().reset_index(drop=True)


In [ ]:
df = df[df['label'] != 'wait'].reset_index(drop=True)

In [ ]:
print(df['input_timestamp'][0])

In [ ]:
DS_LENGTH = len(df['close'])
denoised_input_open = [None]*DS_LENGTH
denoised_input_high = [None]*DS_LENGTH
denoised_input_low = [None]*DS_LENGTH
denoised_input_close = [None]*DS_LENGTH
denoised_input_volume = [None]*DS_LENGTH

for i in range(0, DS_LENGTH):
    denoised_input_open[i] = denoise(df['input_open'][i])
    denoised_input_high[i] = denoise(df['input_high'][i])
    denoised_input_low[i] = denoise(df['input_low'][i])
    denoised_input_close[i] = denoise(df['input_close'][i])
    denoised_input_volume[i] = denoise(df['input_volume'][i])

df['denoised_input_open'] = denoised_input_open
df['denoised_input_high'] = denoised_input_high
df['denoised_input_low'] = denoised_input_low
df['denoised_input_close'] = denoised_input_close
df['denoised_input_volume'] = denoised_input_volume

In [ ]:
ind_list = df.ta.indicators(as_list=True)
indies_partition_size = 10
p_ind_list = [ind_list[i:i + indies_partition_size] for i in range(0, len(ind_list), indies_partition_size)]
ind_columns = []
p_ind_list
exclude_indies = ['above', 'above_value']

In [ ]:
for indi in ind_list:
    if not indi in exclude_indies:
        print(indi)
        indi_result = {}
        new_cols = []
        for i in range(0, DS_LENGTH):
            indi_input = pd.DataFrame()
            indi_input['open'] = df['denoised_input_open'][i].copy()
            indi_input['high'] = df['denoised_input_high'][i].copy()
            indi_input['low'] = df['denoised_input_low'][i].copy()
            indi_input['close'] = df['denoised_input_close'][i].copy()
            indi_input['volume'] = df['denoised_input_volume'][i].copy()
            indi_input['timestamp'] = df['input_timestamp'][i].copy(
            ).reset_index(drop=True)
            indi_input.set_index(pd.DatetimeIndex(
                indi_input["timestamp"]), inplace=True)
            indi_fn = getattr(indi_input.ta, indi)
            data = indi_fn()
            if len(new_cols) == 0:
                if not isinstance(data, pd.Series):
                    new_cols = new_cols + data.columns.to_numpy().tolist()
                else:
                    new_cols = new_cols + [indi]
                for col_name in new_cols:
                    indi_result[col_name] = [None]*DS_LENGTH
                ind_columns = ind_columns + new_cols
            for col_name in new_cols:
                if not isinstance(data, pd.Series):
                    indi_result[col_name][i] = data[col_name]
                else:
                    indi_result[col_name][i] = data
        for col_name in new_cols:
            df[col_name] = indi_result[col_name]
        

In [ ]:
gaf_transformer = GramianAngularField(method='difference', image_size=INPUT_SIZE)
df_gaf_input_open = [None]*DS_LENGTH
df_gaf_input_high = [None]*DS_LENGTH
df_gaf_input_low = [None]*DS_LENGTH
df_gaf_input_close = [None]*DS_LENGTH
df_gaf_input_volume = [None]*DS_LENGTH
for i in range(0, DS_LENGTH):
    if len(df['denoised_input_close'][i]) > 0:
        df_gaf_input_open[i] = gaf_transformer.fit_transform(df['denoised_input_open'][i][-INPUT_SIZE:].reshape(1, -1))
        df_gaf_input_high[i] = gaf_transformer.fit_transform(df['denoised_input_high'][i][-INPUT_SIZE:].reshape(1, -1))
        df_gaf_input_low[i] = gaf_transformer.fit_transform(df['denoised_input_low'][i][-INPUT_SIZE:].reshape(1, -1))
        df_gaf_input_close[i] = gaf_transformer.fit_transform(df['denoised_input_open'][i][-INPUT_SIZE:].reshape(1, -1))
        df_gaf_input_volume[i] = gaf_transformer.fit_transform(df['denoised_input_volume'][i][-INPUT_SIZE:].reshape(1, -1))
df['gaf_open'] = df_gaf_input_open
df['gaf_high'] = df_gaf_input_high
df['gaf_low'] = df_gaf_input_low
df['gaf_close'] = df_gaf_input_close
df['gaf_volume'] = df_gaf_input_volume


In [ ]:
for col_name in ind_columns:
    gaf_col = [None]*DS_LENGTH
    for i in range(0, DS_LENGTH):
        if len(df['denoised_input_close'][i]) > 0:
            gaf_col[i] = gaf_transformer.fit_transform(
                df[col_name][i][-INPUT_SIZE:].to_numpy().reshape(1, -1))
    df[col_name] = gaf_col

In [ ]:
fig = plt.figure(figsize=(10, 5))
grid = ImageGrid(fig, 111, nrows_ncols=(5, 10), axes_pad=0.1, share_all=True,
                 cbar_mode='single')
for i, ax in enumerate(grid):
    image_item = ax.imshow(df['ao'].to_numpy()[i+10].squeeze(), cmap='rainbow', origin='lower', vmin=-1., vmax=1.)
grid[0].get_yaxis().set_ticks([])
grid[0].get_xaxis().set_ticks([])
plt.colorbar(image_item, cax=grid.cbar_axes[0])
ax.cax.toggle_label(True)
fig.suptitle("Gramian angular difference fields for the 50 close price time series in the "
             "'BTCUSDT' dataset")
plt.show()

In [ ]:
for col_name in ind_columns:
    df_train = df[[col_name, 'gaf_open', 'gaf_highs',
                   'gaf_low', 'gaf_close', 'gaf_volume']]
    pat = r'^(.*)_\d+.png'
    images_path = '/kaggle/working/images/'
    indi_cols_partition_size = 5
    ind_columns_parts = [ind_columns[i:i + indi_cols_partition_size]
                        for i in range(0, len(ind_columns), indi_cols_partition_size)]
    if not os.path.exists(images_path):
        os.makedirs(images_path)
    files = get_image_files(images_path)
    for f in files:
        os.remove(f)
    files = get_image_files(images_path)
    print(files)
    L = len(df_train['close'])
    for i in range(0, L):
        i_open = df_train['gaf_open'].to_numpy()[i].squeeze()
        i_high = df_train['gaf_high'].to_numpy()[i].squeeze()
        i_low = df_train['gaf_low'].to_numpy()[i].squeeze()
        i_close = df_train['gaf_close'].to_numpy()[i].squeeze()
        i_volume = df_train['gaf_volume'].to_numpy()[i].squeeze()
        row1 = np.concatenate([i_open, i_high, i_low])
        row2 = np.concatenate([i_close, i_volume, col_name])
        row3 = row2
        #indi_rows = []
        #for part in ind_columns_parts:
        #    i_data = [df_train[col_name].to_numpy()[i].squeeze() for col_name in part]
        #    if len(i_data) < indi_cols_partition_size:
        #        for j in range(0, indi_cols_partition_size - len(i_data)):
        #            i_data = i_data + [i_volume]
        #    indi_rows = indi_rows + [np.concatenate(i_data)]
        #all_rows = [row1] + indi_rows
        data = np.concatenate([row1, row2, row3], axis=1)
        label = df_train['label'].to_numpy()[i]
        matplotlib.image.imsave(images_path + label + '_' + str(i) + '.png', data)
    files = get_image_files(images_path)
    dls = ImageDataLoaders.from_name_re(images_path, files, pat)
    dls.show_batch()
    learn = vision_learner(dls, resnet101, metrics=error_rate)  # resnet152
    #learn.lr_find()
    learn.fine_tune(10, 0.0002290867705596611)


In [ ]:

#learn.show_results()
#learn.predict(files[0])

In [ ]:
#learn.export(fname='/kaggle/working/model.pkl')
